In [1]:
import logging
import os
import random
import time

import mne
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader
from torcheeg import transforms
from torcheeg.datasets import MNEDataset
from torcheeg.models import TSCeption
from torcheeg.trainers import ClassificationTrainer
import Raw_EEG_processing



In [2]:
dataset = Raw_EEG_processing.load_data(subject_id_list=list(range(1,10)))

Extracting EDF parameters from /Users/jakesolsky/Documents/GitHub/EEG-AI-Control-System/files/S001/S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['B']
Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 801 original time points ...
1 bad epochs dropped
Extracting EDF parameters from /Users/jakesolsky/Documents/GitHub/EEG-AI-Control-System/files/S001/S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['B']
Not setting metadata
1 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Usi

['data.mdb']

In [36]:
os.makedirs('./tmp_out/examples_mne_dataset/log', exist_ok=True)
logger = logging.getLogger('Examples of MNEDataset')
logger.setLevel(logging.DEBUG)
console_handler = logging.StreamHandler()
timeticks = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
file_handler = logging.FileHandler(
    os.path.join('./tmp_out/examples_mne_dataset/log', f'{timeticks}.log'))
logger.addHandler(console_handler)
logger.addHandler(file_handler)

In [37]:


def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(42)

In [38]:
class MyClassificationTrainer(ClassificationTrainer):
    def log(self, *args, **kwargs):
        if self.is_main:
            logger.info(*args, **kwargs)
    

In [39]:
"""metadata_list = [{
    'subject': subject_id,
    'run': run_id
} for subject_id in range(1, 22)
                 for run_id in [6, 10, 14]]  # motor imagery: hands vs feet

epochs_list = []
for metadata in metadata_list:
    physionet_path = mne.datasets.eegbci.load_data(metadata['subject'],
                                                   metadata['run'],
                                                   update_path=False)[0]
    raw = mne.io.read_raw_edf(physionet_path, preload=True, stim_channel='auto')
    mne.datasets.eegbci.standardize(raw)

    montage = mne.channels.make_standard_montage('standard_1005')
    raw.set_montage(montage)

    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')
    events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))
    picks = mne.pick_types(raw.info,
                           meg=False,
                           eeg=True,
                           stim=False,
                           eog=False,
                           exclude='bads')
    # init Epochs with raw EEG signals and corresponding event annotations. Here, tmin is set to -1., and tmax is set to 4.0, to avoid classification of evoked responses by using epochs that start 1s after cue onset.
    epochs_list.append(
        mne.Epochs(raw,
                   events,
                   dict(hands=2, feet=3),
                   tmin=-1.,
                   tmax=4.0,
                   proj=True,
                   picks=picks))"""

"metadata_list = [{\n    'subject': subject_id,\n    'run': run_id\n} for subject_id in range(1, 22)\n                 for run_id in [6, 10, 14]]  # motor imagery: hands vs feet\n\nepochs_list = []\nfor metadata in metadata_list:\n    physionet_path = mne.datasets.eegbci.load_data(metadata['subject'],\n                                                   metadata['run'],\n                                                   update_path=False)[0]\n    raw = mne.io.read_raw_edf(physionet_path, preload=True, stim_channel='auto')\n    mne.datasets.eegbci.standardize(raw)\n\n    montage = mne.channels.make_standard_montage('standard_1005')\n    raw.set_montage(montage)\n\n    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')\n    events, _ = mne.events_from_annotations(raw, event_id=dict(T1=2, T2=3))\n    picks = mne.pick_types(raw.info,\n                           meg=False,\n                           eeg=True,\n                           stim=False,\n                       

In [40]:
"""dataset = MNEDataset(epochs_list=epochs_list,
                     metadata_list=metadata_list,
                     chunk_size=160,
                     overlap=80,
                     io_path='./tmp_out/examples_mne_dataset/physionet',
                     offline_transform=transforms.Compose(
                         [transforms.MeanStdNormalize(),
                          transforms.To2d()]),
                     online_transform=transforms.ToTensor(),
                     label_transform=transforms.Compose([
                         transforms.Select('event'),
                         transforms.Lambda(lambda x: x - 2)
                     ]),
                     num_worker=2)"""

"dataset = MNEDataset(epochs_list=epochs_list,\n                     metadata_list=metadata_list,\n                     chunk_size=160,\n                     overlap=80,\n                     io_path='./tmp_out/examples_mne_dataset/physionet',\n                     offline_transform=transforms.Compose(\n                         [transforms.MeanStdNormalize(),\n                          transforms.To2d()]),\n                     online_transform=transforms.ToTensor(),\n                     label_transform=transforms.Compose([\n                         transforms.Select('event'),\n                         transforms.Lambda(lambda x: x - 2)\n                     ]),\n                     num_worker=2)"

In [41]:
"""if __name__ == "__main__":
    dataset = MNEDataset(epochs_list=epochs_list,
                  metadata_list=metadata_list,
                  chunk_size=160,
                  overlap=80,
                  io_path='./tmp_out/examples_mne_dataset/physionet',
                  offline_transform=transforms.Compose(
                      [transforms.MeanStdNormalize(),
                       transforms.To2d()]),
                  online_transform=transforms.ToTensor(),
                  label_transform=transforms.Compose([
                      transforms.Select('event'),
                      transforms.Lambda(lambda x: x - 2)
                  ]),
                  io_mode='pickle',
                  num_worker=2)"""

'if __name__ == "__main__":\n    dataset = MNEDataset(epochs_list=epochs_list,\n                  metadata_list=metadata_list,\n                  chunk_size=160,\n                  overlap=80,\n                  io_path=\'./tmp_out/examples_mne_dataset/physionet\',\n                  offline_transform=transforms.Compose(\n                      [transforms.MeanStdNormalize(),\n                       transforms.To2d()]),\n                  online_transform=transforms.ToTensor(),\n                  label_transform=transforms.Compose([\n                      transforms.Select(\'event\'),\n                      transforms.Lambda(lambda x: x - 2)\n                  ]),\n                  io_mode=\'pickle\',\n                  num_worker=2)'

In [42]:
#input individual subject in data import at the top (0-108)

#This is outer mne dataset object
print(dataset)
#This is Trial (0-347 (?))
print(dataset[1])
#This is the sensor channel [0-347][0][0-63]
print(len(dataset[0][0][0]))
print('\n\n')
#This is the individual trial's metadata [0-347][1]
print(dataset[0][1])
#This is the time series [0-348][0][0-800]
print(len(dataset[0][0][0]))
print(dataset[0][0][1])
print(dataset[0][0][2])
print(dataset[0][1])
print(dataset[1][0])
print(dataset[108][1])

MNEDataset(
    io_path='./io/mne',
    io_size=10485760,
    io_mode='pickle',
    in_memory=False,
    chunk_size=-1,
    overlap=0,
    num_channel=-1,
    online_transform=None,
    offline_transform=None,
    label_transform=None,
    before_trial=None,
    after_trial=None,
    num_worker=1,
    verbose=True
)
length=348


FileNotFoundError: [Errno 2] No such file or directory: './io/mne/eeg/2_1'

In [ ]:
from torcheeg.model_selection import KFold

k_folds = KFold(n_splits=5,
                split_path=f'./tmp_out/split',)



In [ ]:
print(k_folds)

KFold(n_splits=5, shuffle=False, random_state=None, split_path='./tmp_out/split')


In [ ]:


for i, (train_dataset, val_dataset) in enumerate(k_folds.split(dataset)):
    print(i, (train_dataset, val_dataset))
    # Initialize the model
    model = TSCeption(num_electrodes=64,
                      num_classes=2,
                      num_T=15,
                      num_S=15,
                      in_channels=1,
                      hid_channels=32,
                      sampling_rate=160,
                      dropout=0.5)

    # Initialize the trainer and use the 0-th GPU for training, or set device_ids=[] to use CPU
    trainer = MyClassificationTrainer(model=model,
                                      lr=1e-4,
                                      weight_decay=1e-4,
                                      device_ids=[])

    # Initialize several batches of training samples and test samples
    train_loader = DataLoader(train_dataset,
                              batch_size=256,
                              shuffle=True,
                              num_workers=4)
    val_loader = DataLoader(val_dataset,
                            batch_size=256,
                            shuffle=False,
                            num_workers=4)
    
    print(trainer + '\n')
    print(train_loader + '\n')
    print(val_loader + '\n')

    # Do 50 rounds of training
    trainer.fit(train_loader, val_loader, num_epochs=50)
    trainer.test(val_loader)
    trainer.save_state_dict(f'./tmp_out/examples_mne_dataset/weight/{i}.pth')


NameError: name 'k_folds' is not defined